<img src="img/Marca-ITBA-Color-ALTA.png" width="200">

# Programación para el Análisis de Datos

## Clase 4 - parte 2 - Limpieza y preprocesamiento de datos

#### Referencias y bibliografía de consulta:

- Python for Data Analysis by Wes McKinney (O’Reilly) 2022 - capítulo 7
- Introduction to Machine Learning with Python by Andreas C. Müller and Sarah Guido (O’Reilly) 2017 - capítulo 4

### Introducción

Generalmente, los datos no vienen limpios y listos para entrenar modelos. Lo más frecuente es que el trabajo de recolectar, procesar, limpiar y transformar los datos, de acuerdo a los objetivos de nuestro proyecto, representen un porcentaje muy elevado del tiempo de trabajo en el proyecto. Depende mucho del proyecto, pero se estima que estas tareas representan aproximadamente un 80% del tiempo de un proyecto. 

En la sección de **limpieza de datos** de este módulo vamos a aprender a detectar y a subsanar a los principales problemas que se suelen encontrar en los dataset:

- **Datos duplicados**
- **Datos faltantes**
- **Valores extremos (outliers)**

Otra tarea importante de la limpieza de datos es la de resolver problemas de formato. Ya vimos algunas formas de resolver estos problemas en los primeros módulos de `Pandas`.

Además, en la sección de **procesamiento**, vamos a ver cómo trabajar con variables categóricas. La mayoría de los modelos solamente puede recibir valores numéricos como input, y por lo tanto vamos a tener que transformar a las variables categóricas en variables numéricas. En esta clase vamos a ver cómo.


### 1) Limpieza de datos

Vamos a comenzar importando `Numpy` y `Pandas` con sus alias, como de costumbre. Vamos a importar también un dataset de ejemplo con información sobre vinos.

In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_csv('data/winemag-data-130k-v2.csv', index_col=0)

dataset.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [3]:
dataset.shape

(65499, 13)

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 65499 entries, 0 to 65498
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                65467 non-null  object 
 1   description            65499 non-null  object 
 2   designation            46588 non-null  object 
 3   points                 65499 non-null  int64  
 4   price                  60829 non-null  float64
 5   province               65467 non-null  object 
 6   region_1               54744 non-null  object 
 7   region_2               25170 non-null  object 
 8   taster_name            51856 non-null  object 
 9   taster_twitter_handle  49467 non-null  object 
 10  title                  65499 non-null  object 
 11  variety                65499 non-null  object 
 12  winery                 65499 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 7.0+ MB


#### 1.1) Detección y filtrado de valores duplicados

`Pandas` provee métodos para detectar y filtrar datos duplicados. 

- `duplicated()` identificando los datos duplicados. Devuelve valores booleanos. 
-  `drop_duplicates()` filtra los casos duplicados 

In [5]:
dataset.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
65494    False
65495    False
65496    False
65497    False
65498    False
Length: 65499, dtype: bool

In [6]:
help('pandas.DataFrame.duplicated')

Help on function duplicated in pandas.DataFrame:

pandas.DataFrame.duplicated = duplicated(self, subset: 'Hashable | Sequence[Hashable] | None' = None, keep: 'DropKeep' = 'first') -> 'Series'
    Return boolean Series denoting duplicate rows.

    Considering certain columns is optional.

    Parameters
    ----------
    subset : column label or sequence of labels, optional
        Only consider certain columns for identifying duplicates, by
        default use all of the columns.
    keep : {'first', 'last', False}, default 'first'
        Determines which duplicates (if any) to mark.

        - ``first`` : Mark duplicates as ``True`` except for the first occurrence.
        - ``last`` : Mark duplicates as ``True`` except for the last occurrence.
        - False : Mark all duplicates as ``True``.

    Returns
    -------
    Series
        Boolean series for each duplicated rows.

    See Also
    --------
    Index.duplicated : Equivalent method on index.
    Series.duplicated : Equ

In [7]:
dataset.duplicated()[dataset.duplicated()==True]

2408     True
2409     True
2410     True
2412     True
2413     True
         ... 
65467    True
65468    True
65475    True
65476    True
65477    True
Length: 2660, dtype: bool

Con el parámetro `keep` podemos determinar cuál de los valores duplicados marcar como duplicados:

In [8]:
dataset.duplicated(keep='last')[dataset.duplicated(keep='last')==True]

41       True
42       True
45       True
46       True
47       True
         ... 
63286    True
63287    True
63288    True
63289    True
64002    True
Length: 2660, dtype: bool

El método `pd.duplicated()` devuelve valores booleanos. Para saber si y cuántos duplicados hay, podemos sumar estos valores. Los `True` van a sumar 1 y los `False` van a sumar 0.  

In [9]:
dataset.duplicated().sum()

2660

Hay 2.660 valores duplicados.

Podemos ajustar el criterio y considerar solamente un subset de columnas para identificar los duplicados. Al especificar un subset, 2 filas serán consideradas duplicados si los valores de dichas columnas están duplicadas, independientemente de los valores de las demás. 

In [10]:
dataset.duplicated(['winery','variety', 'title', 'taster_name']).sum()

2910

Vemos que el número de duplicados se incrementó a 2.910.

##### drop_duplicates()

In [11]:
dataset = dataset.drop_duplicates(['winery','variety', 'title', 'taster_name'],
                                   keep='last')

In [12]:
dataset.shape

(62589, 13)

#### 1.2) Detección, filtrado e imputación de datos faltantes

Es frecuente que en los datasets haya **datos faltantes**. Puede ser por algún error en el proceso de carga o recolección de los datos. También puede ser que la información provenga de diferentes fuentes y en alguna de las fuentes no haya datos para alguna observación. 

En cualquier caso, uno de los objetivos de `Pandas` es hacer que el trabajo con los datos faltantes sea lo menos doloroso posible. Por ejemplo, todas las estadísticas descriptivas de los objetos `Pandas` ignoran por default a los datos que faltan.

La principales formas que tiene `Pandas` de representar a los datos nulos son las siguentes:

- `None`: objeto de Python que representa ausencia de dato
- `NaN` (Not a Number): representación de datos faltantes para datos numéricos
- `NaT`: se utiliza para valores faltantes del tipo Timestamp

##### Detección de datos faltantes

Los objetos de `Pandas` tienen el método `isnull()` que nos permite identificar datos faltantes. 

Veamos si nuestro dataset tiene datos faltantes:

In [13]:
dataset.isnull()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,False,False,False,False,True,False,False,True,False,False,False,False,False
1,False,False,False,False,False,False,True,True,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,True,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65494,False,False,False,False,False,False,False,True,False,False,False,False,False
65495,False,False,True,False,False,False,False,True,False,False,False,False,False
65496,False,False,False,False,False,False,False,False,True,True,False,False,False
65497,False,False,False,False,False,False,False,False,True,True,False,False,False


Como el método `isnull()` devuelve valores boolenanos, frecuentemente es más cómodo aplicar el método `sum()` al output, de modo tal de tener un conteo de los datos faltantes por columna. 

Recordemos que el método `sum()` tomará como `1` a los valores `True` y como `0` a los `False`.  

In [14]:
dataset.isnull().sum()

country                     32
description                  0
designation              18136
points                       0
price                     4489
province                    32
region_1                 10277
region_2                 38442
taster_name              13176
taster_twitter_handle    15455
title                        0
variety                      0
winery                       0
dtype: int64

##### Filtrado de datos faltantes: `dropna()`

`Pandas` nos provee de un método para identificar y filtrar datos faltantes en un solo paso. Este es el método `dropna()`.

Apliquémoslo a nuestro dataset con datos nulos.

In [15]:
dataset_dropna = dataset.dropna()

Vamos a comparar los `shape` del dataset con nulos y el dataset con los nulos filtrados, para ver el resultado del filtro:

In [16]:
print(dataset.shape)
print(dataset_dropna.shape)

(62589, 13)
(10516, 13)


De las 62.589 observaciones (filas), nos quedaron 10.516. Esto sucede porque el argumento `how` viene por default con el valor `'any'`. Esto quiere decir que es suficiente con que haya 1 valor nulo en la fila para que ésta sea filtrada. El análisis se hace por fila porque también por default `axis=0`. Podemos pedir que el análisis sea por columna cambiando el valor a 1.

Como se pueden imaginar, filtrar toda la fila por tener un dato nulo puede representar un problema. Veamos cómo podemos ajustar el criterio:

In [17]:
dataset_dropna_fullrow = dataset.dropna(how = 'all')

Ahora le pedimos al método que solamente filtre la fila si todos los datos son nulos. Veamos el resultado:

In [18]:
print(dataset.shape)
print(dataset_dropna_fullrow.shape)

(62589, 13)
(62589, 13)


No filtramos nada. Tal vez `how= 'all'` es un criterio demasiado restrictivo. Veamos si podemos establecer un criterio más flexible:

In [19]:
# Tolero como máximo 4 columnas con datos nulos por fila:
umbral = dataset.shape[1] - 4
dataset_dropna_tresh = dataset.dropna(axis = 0, thresh=umbral)
print(dataset.shape)
print(dataset_dropna_tresh.shape)

(62589, 13)
(62282, 13)


Con el uso del argumento `thresh`(de umbral) puedo definir un criterio que se amolde a las necesidades de mi análisis.

También podemos usar el argumento `subset` para filtrar considerando solamente un subconjunto de columnas.

In [20]:
dataset_dropna_subset = dataset.dropna(subset=['price', 'province'])
print(dataset.shape)
print(dataset_dropna_subset.shape)

(62589, 13)
(58072, 13)


In [21]:
dataset.shape[0] - dataset_dropna_subset.shape[0]

4517

Vemos que, como por default, el argumento `how = any`, entonces se eliminan las filas que tengan un nulo en por lo menos una de las columnas. Podemos modificar esto para que todas las columnas del subset sean nulas para que el registro sea eliminado. 

In [22]:
dataset_dropna_subset = dataset.dropna(how = 'all', subset=['price', 'province'])
print(dataset.shape)
print(dataset_dropna_subset.shape)

(62589, 13)
(62585, 13)


In [23]:
dataset.shape[0] - dataset_dropna_subset.shape[0]

4

#### Imputación

Una alternativa a filtrar los datos es completar el valor que falta con algún valor que podamos estimar o definir de acuerdo a algún criterio. La técnica de completar los datos faltantes se llama **imputación**.

`Pandas` provee un método para completar los datos faltantes: `fillna()`. Como primer paso, imputamos a cada dato faltante por el valor promedio de la columna:

In [25]:
dataset['price'][:5]

0     NaN
1    15.0
2    14.0
3    13.0
4    65.0
Name: price, dtype: float64

In [26]:
dataset['price'].mean()

35.32779690189329

In [27]:
dataset['price'].fillna(dataset['price'].mean())[:5]

0    35.327797
1    15.000000
2    14.000000
3    13.000000
4    65.000000
Name: price, dtype: float64

También podemos usar el `groupby` para dividir el dataset en grupos e imputar por la media de estos grupos. 

En este caso, vamos a asumir que una distinción relevante es calcular la media agrupando por variety. 

In [28]:
# Usamos groupby para imputar:
dataset[['variety', 'price']].groupby('variety').transform('mean')

,price
0,22.677998
1,25.304428
2,23.211852
3,31.284515
4,47.720209
...,...
65494,35.152988
65495,47.007251
65496,20.341443
65497,25.351351


In [29]:
dataset = dataset.fillna(dataset[['variety', 'price']].groupby('variety').transform('mean'))

Veamos cuántos datos nulos quedan luego de realizar la transformación:

In [30]:
dataset.isnull().sum()

country                     32
description                  0
designation              18136
points                       0
price                       10
province                    32
region_1                 10277
region_2                 38442
taster_name              13176
taster_twitter_handle    15455
title                        0
variety                      0
winery                       0
dtype: int64

Siguen quedando 10 filas con la columna 'price' nula. Analicemos qué está pasando.

Usamos la columna 'variety' para agregar. Veamos cuántas variedades diferentes tenemos:

In [31]:
len(dataset['variety'].unique())

590

Cuantos registros de cada variedad hay?

In [32]:
dataset['variety'].value_counts()

variety
Pinot Noir                  6380
Chardonnay                  5617
Cabernet Sauvignon          4603
Red Blend                   4366
Bordeaux-style Red Blend    3313
                            ... 
Irsai Oliver                   1
Baga-Touriga Nacional          1
Muscatel                       1
Viognier-Grenache Blanc        1
Misket                         1
Name: count, Length: 590, dtype: int64

Vemos que hay muchas variedades con un único registro. Es posible que muchas tengan todos valores nulos. 

In [33]:
dataset[['variety', 'price']].groupby('variety').mean().isnull().sum()

price    8
dtype: int64

Hay 8 variedades para las cuales no tenemos ningun precio. Vamos a terminar la imputación aplicando la media general del dataset:

In [34]:
dataset['price'] = dataset['price'].fillna(dataset['price'].mean())

dataset.isnull().sum()

country                     32
description                  0
designation              18136
points                       0
price                        0
province                    32
region_1                 10277
region_2                 38442
taster_name              13176
taster_twitter_handle    15455
title                        0
variety                      0
winery                       0
dtype: int64

La **imputación** de datos faltantes es un tema amplio y existen diferentes técnicas. No es necesario imputar siempre por la media, por ejemplo. Se puede imputar usando los métodos de `Pandas` pero  también hay que tener en cuenta que hay disponibles librerías muy útiles que simplifican la tarea de la imputación. 

Por ejemplo, `Sklearn`, una biblioteca de Python que provee implementaciones de algoritmos de machine learning ofrece la clase `SimpleImputer`. Dejamos el link, pero como todavía no aprendimos a trabajar con `Sklearn`, no vamos a entrar en detalle ahora:

https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer

#### 1.3) Detectar y filtrar outliers

No existe un criterio que sea válido en todos los casos para identificar los outliers. 

En esta práctica consideraremos como outliers a los datos que sean mayores al tercer cuartil más 1.5 veces el rango intercuartil o menores al primer cuartil menos 1.5 veces el rango intercuartil. 

Vamos a generar una máscaras booleanas que nos van a servir para identificar a los outliers.

In [35]:
dataset.loc[:, 'price'].max()

2500.0

In [36]:
dataset.loc[:, 'price'].quantile(0.5)

26.0

In [37]:
q1 = dataset.loc[:, 'price'].quantile(0.25)
q3 = dataset.loc[:, 'price'].quantile(0.75)
iqr = q3 - q1

In [38]:
q1

18.0

In [39]:
q3

44.471774193548384

In [40]:
# Rango intercuartil
iqr

26.471774193548384

Una forma frecuente de considerar a valores como outliers, es si superan el Q3 + 1.5 * RI o Q1 - 1.5 * RI.

Este es el criterio que por ejemplo se utiliza en los box-plots.

In [41]:
q3 + 1.5 * iqr

84.17943548387096

In [42]:
q1 - 1.5 * iqr

-21.707661290322577

Vemos que el valor Q1 - 1.5 * RI es negativo, por lo cual, para este caso de uso, carece de sentido, y por lo tanto no lo vamos a considerar. 

In [43]:
mask_high = (dataset.loc[:, 'price'] > (q3 + 1.5 * iqr))

In [44]:
mask_high

0        False
1        False
2        False
3        False
4        False
         ...  
65494    False
65495    False
65496    False
65497    False
65498    False
Name: price, Length: 62589, dtype: bool

In [45]:
dataset.loc[mask_high,:]

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
60,US,"Syrupy and dense, this wine is jammy in plum a...",Estate,86,100.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Okapi 2013 Estate Cabernet Sauvignon (Napa Val...,Cabernet Sauvignon,Okapi
111,US,This 100% varietal wine opens in a heady aroma...,Wolff Vineyard,87,85.0,California,Yountville,Napa,Virginie Boone,@vboone,Piña 2013 Wolff Vineyard Cabernet Sauvignon (Y...,Cabernet Sauvignon,Piña
139,France,"Beautiful deep gold color. Intense, concentrat...",Cuvée Jerémy Sélection de Grains Nobles,90,112.0,Alsace,Alsace,NaN,NaN,NaN,Kuentz-Bas 2007 Cuvée Jerémy Sélection de Grai...,Pinot Gris,Kuentz-Bas
144,US,"Thick and brooding, this dark, sweetly tannic ...",K Block,91,85.0,California,Spring Mountain District,Napa,Virginie Boone,@vboone,Terra Valentine 2013 K Block Cabernet Sauvigno...,Cabernet Sauvignon,Terra Valentine
236,US,"Shows rich, firm mountain tannins and well-rip...",Kiss Ridge Vineyard,85,85.0,California,Diamond Mountain District,Napa,NaN,NaN,Meeker 2004 Kiss Ridge Vineyard Cabernet Sauvi...,Cabernet Sauvignon,Meeker
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65357,France,This most powerful of Grand Cru vineyards has ...,NaN,96,496.0,Burgundy,Chambertin,NaN,Roger Voss,@vossroger,Roche de Bellene 2011 Chambertin,Pinot Noir,Roche de Bellene
65359,France,Spice and mint dominate this intensely aromati...,NaN,95,295.0,Burgundy,Clos de la Roche,NaN,Roger Voss,@vossroger,Roche de Bellene 2011 Clos de la Roche,Pinot Noir,Roche de Bellene
65363,France,With just a touch of bottle age giving a toast...,Comtes de Champagne Blanc de Blancs Brut,95,170.0,Champagne,Champagne,NaN,Roger Voss,@vossroger,Taittinger 2005 Comtes de Champagne Blanc de B...,Chardonnay,Taittinger
65364,France,Rich black fruits are supported by a dark stru...,NaN,95,262.0,Burgundy,Echézeaux,NaN,Roger Voss,@vossroger,Roche de Bellene 2011 Echézeaux,Pinot Noir,Roche de Bellene


Hay que tener mucho cuidado con el filtrado o la corrección de valores extremos. Antes de realizar una modificación en los datos tenemos que estar seguros que esos valores extremos en realidad no sean información valiosa.

En el caso de este dataset, lo más probable es que los vinos con precio muy alto sean simplemente vinos de alta gama. De todos modos, el caso de querer corregir valores extremos, podríamos realizarlo del siguiente modo:

In [46]:
dataset.loc[mask_high, 'price'] = dataset.loc[mask_high, 'price'].apply(lambda x: (q3 + 1.5 * iqr))

In [47]:
dataset['price'].max()

84.17943548387096

Otro criterio que podríamos usar para detectar valores extremos es buscar valores que están a más de 3 desvíos estándar de la media. 

### 2) Variables categóricas 


La forma más común de representar las variables categóricas es el uso de la codificación $one-hot-encoding$ también conocida como `variables dummy`. La idea detrás de las variables ficticias es reemplazar una variable categórica con una o más nuevas características que pueden tener los valores 0 y 1. 

Veamos un ejemplo tomado del manual de Andreas C. Müller and Sarah Guido:

<img src="img/one_hot.png" width="350">


`Pandas` cuenta con el método `pd.get_dummies()` que recibe una serie o una lista de series y realiza el one-hot-encoding.

Recordemos que una variable con `k` categorías se puede representar con `k-1` variables.

Por eso un parámetro clave de `pd.get_dummies()` es `drop_first = True` que genera `k-1` categorías en lugar de `k`. 

In [48]:
pd.get_dummies(dataset['country'])

,Argentina,Armenia,Australia,Austria,Bosnia and Herzegovina,Brazil,Bulgaria,Canada,Chile,Croatia,...,Serbia,Slovakia,Slovenia,South Africa,Spain,Switzerland,Turkey,US,Ukraine,Uruguay
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65494,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
65495,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
65496,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
65497,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


Podemos agregar un prefijo para identificar la categoría:

El método `pd.get_dummies()` genera como output un `DataFrame`. Vamos a almacenar este objeto en la variable `dummies`. Vamos a pasar también el argumento `drop_first=True`.

In [49]:
dummies = pd.get_dummies(dataset['country'], prefix='country', \
                         drop_first=True)
dummies

,country_Armenia,country_Australia,country_Austria,country_Bosnia and Herzegovina,country_Brazil,country_Bulgaria,country_Canada,country_Chile,country_Croatia,country_Cyprus,...,country_Serbia,country_Slovakia,country_Slovenia,country_South Africa,country_Spain,country_Switzerland,country_Turkey,country_US,country_Ukraine,country_Uruguay
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65494,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
65495,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
65496,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
65497,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


Podemos generar un dataset con dummies directamente pasando un `DataFrame` con las variables numéricas y las categoricas. `Pandas` va a ignorar las variables numéricas. Además, por ejecutar la función sobre un `DataFrame`, `Pandas` va a agregar automáticamente el prefijo usando el nombre de la columna.

In [50]:
pd.get_dummies(dataset[['province', 'country', 'price', 'points']], drop_first=True)

,price,points,province_Aconcagua Costa,province_Aconcagua Valley,province_Aegean,province_Agioritikos,province_Ahr,province_Alenquer,province_Alentejano,province_Alentejo,...,country_Serbia,country_Slovakia,country_Slovenia,country_South Africa,country_Spain,country_Switzerland,country_Turkey,country_US,country_Ukraine,country_Uruguay
0,22.677998,87,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,15.000000,87,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,14.000000,87,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,13.000000,87,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,65.000000,87,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65494,45.000000,90,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
65495,22.000000,90,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
65496,20.000000,90,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
65497,31.000000,90,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


### 3) Expresiones Regulares

Una **expresión regular** es una secuencia de caracteres que define un patrón de búsqueda de texto. Las regex constituyen lenguaje muy flexible que sirve para identificar y extraer información de un cuerpo de caracteres no estructurado.

**Referencia:** https://regex101.com/

In [51]:
with pd.option_context('display.max_colwidth', 10000):
    display(dataset['title'].sample(10, random_state=20))

39400                             29 & Oak Wines 2013 Reserve Cabernet Sauvignon (Oakville)
37091                          Seven Falls 2015 GPS Jones Vineyard Viognier (Wahluke Slope)
8823                                                       Château Reysson 2011  Haut-Médoc
49001                                    Cantina Cortaccia 2014 Kofl Sauvignon (Alto Adige)
63033                                                   Arrayán 2007 Premium Red (Mentrida)
58642                                       V. Sattui 2013 Reserve Chardonnay (Napa Valley)
25381    Williams Selyem 2012 Burt Williams' Morning Dew Ranch Pinot Noir (Anderson Valley)
18586                        Herdade dos Templários 2011 Vale das Donas Branco White (Tejo)
48520                             Lantides 2014 Ancient Varietal Moschofilero (Peloponnese)
12860                                            Wiese & Krohn 1964 Branco Colheita  (Port)
Name: title, dtype: object

Vamos a crear una expresión regular simple para extraer el año del texto de título.

In [52]:
import re

pattern = r"\b([12]\d{3})\b"
r = re.compile(pattern)

Este patrón busca y extrae años de cuatro dígitos (probablemente entre 1000-2999)

    -  \b: Asegura que el año es un número completo (no parte de otro más largo)
    - ([12]\d{3}): Captura un número de 4 dígitos que empieza con '1' o '2'


Es útil para encontrar años de cosecha en texto, como "Vino (2013) Reserva", extrayendo "2013".

In [53]:
type(r)

re.Pattern

In [54]:
s1 = '29 & oak wines 2013 reserve cabernet sauvignon (oakville)'

In [55]:
match = r.search(s1)
match

<re.Match object; span=(15, 19), match='2013'>

In [56]:
match.groups()

('2013',)

In [57]:
match[0]

'2013'

¿Qué sucedería si tuviéramos más de una ocurrencia? Vamos a combinar 2 títulos para tener 2 años. 

In [58]:
s2 = '''
seven falls 2015 gps jones vineyard viognier (wahluke slope). 
v. sattui 2013 reserve chardonnay (napa valley)
'''

In [59]:
match = r.search(s2)
match

<re.Match object; span=(13, 17), match='2015'>

In [60]:
match.groups()

('2015',)

El método `search()` nos trae solamente la primera ocurrencia. Para obtener todas podemos usar el método `findall()`

In [61]:
r.findall(s2)

['2015', '2013']

In [62]:
dataset['year'] = dataset['title'].apply(lambda x: r.search(x)[0] if isinstance(x, str) and r.search(x) else None)

In [70]:
dataset[['title', 'year']].sample(10)

,title,year
22816,Occasio 2011 Thatcher Bay Vineyard Merlot (Liv...,2011
22560,Keuka Spring 2015 Dynamite Vineyard Gewürztram...,2015
23438,Domaine Rossignol-Trapet 2011 Chambertin,2011
1925,Virna Borgogno 2009 Cannubi Boschis (Barolo),2009
45291,Kent Rasmussen 2007 Pinot Noir (Carneros),2007
35184,Ad Vivum 2011 Sleeping Lady Vineyard Cabernet ...,2011
52785,Château Hosanna 2008 Pomerol,2008
61392,Fontes da Cunha 2015 Quinta do Mondego Branco ...,2015
41002,Hogue 2012 Genesis Syrah (Columbia Valley (WA)),2012
57488,Conway 2009 Cabernet Sauvignon (Rutherford),2009


In [71]:
dataset['year'].dtype

dtype('O')

In [72]:
dataset.loc[dataset['year'].isnull(), ['title', 'year']]

,title,year
63,Roland Champion NV Brut Rosé (Champagne),None
69,Collet NV Brut Rosé (Champagne),None
237,Consorzio Vini Tipici di San Marino NV Moscato...,None
326,Marsuret NV Extra Dry (Prosecco di Valdobbiad...,None
332,Sommariva NV Palazzo Rosso Brut (Prosecco di ...,None
...,...,...
65261,Vicente Gandia NV Sandara Sparkling (Spain),None
65274,Ca' Momi NV Ca' Secco Sparkling (California),None
65351,Honora NV Cabernet Sauvignon (America),None
65426,Cairdeas NV Tri Red (Columbia Valley (WA)),None


In [73]:
dataset['year'].isnull().sum()

2090

Hay 2.090 vinos que son NV, es decir nonvintage, que normalmente son blends de dos o más años. Podríamos imputar como 'NV' y dejar a year como una variable categórica o pasar a int y buscar otra forma de imputar a los NV. La estrategia a utilizar dependerá de nuestro proyecto.

In [74]:
dataset['year'] = dataset['year'].fillna('NV')

In [75]:
dataset[['title', 'year']].sample(15)

,title,year
11809,CK Mondavi 2014 Willow Springs Pinot Grigio (C...,2014
58635,El Pistolero 2014 Limitado Chardonnay (Santa L...,2014
40165,Kaleidos 2014 Reserve Syrah (Paso Robles),2014
45257,Mas Saint-Louis 2013 White (Châteauneuf-du-Pape),2013
54721,Alex Gambal 2013 Maltroie Premier Cru (Chassa...,2013
10156,Leonard Kreusch 2014 Piesporter Michelsberg Ka...,2014
3701,Clos de Chacras 2010 Gran Estirpe Red (Mendoza),2010
30802,Joseph Drouhin 2012 Domaine Drouhin-Vaudon Séc...,2012
25608,J. Portugal Ramos 2014 Vila Santa Reserva Bran...,2014
16183,Domaine Zind-Humbrecht 2015 Roche Calcaire Gew...,2015


## Sintaxis en expresiones regulares en Python


#### Metacaracteres

    .  símbolo que indica cualquier caracter con excepción de nueva línea (\n)
    ^  símbolo que indica comienzo del string
    $  símbolo que indica fin del string
    \  símbolo que escapa caracteres reservados
    |  o lógico 
    \d símbolo que indica cualquier dígito del 0 al 9
    \w símbolo que indica cualquier caracter alfanumérico (A-Z, a-z, 0-9 y _)
    \s símbolo que indica cualquier espacio en blanco (espacio, tabulado, nueva línea, etc.)
    \D símbolo que indica cualquier caracter que no sea dígito
    \W símbolo que indica cualquier caracter que no sea alfanumérico
    \S símbolo que indica cualquier caracter que no sea espacio en blanco

        
#### Cuantificadores

    *  símbolo que indica cero o más ocurrencias
    +  símbolo que indica una o más ocurrencias
    ?  símbolo que indica optativo (puede estar o no)
    {m}  donde m es un número entero, indica exactamente m repeticiones 
    {m,n}  donde m y n son números enteros, indica al menos m repeticiones y como máximo n repeticiones

#### Opciones 

    [abc]  indica un caracter perteneciente al conjunto de valores posibles especificado entre corchetes
    [a-z]  indica un caracter perteneciente al intervalo de valores posibles especificado entre corchetes

#### Grupos 
    ( )  define un grupo
    (?P<group_name> ) define un grupo etiquetado
    
#### Ejemplos de sintaxis:

`\d+` encuentra números de un dígito o más

`.*` encuentra cadenas de caracteres de cualquier longitud, incluso vacías.

`\w{2,6}` encuentra un conjunto de caracteres alfanuméricos con una longitud que va de 2 a 6 caracteres

`[a-zA-Z]` encuentra lo mismo que `[a-zA-Z]{1}` que es cualquier caracter entre a y z minúscula o A y Z mayúscula

`[a-zA-Z]+` encuentra cadenas de caracteres con letras entre a y z y A y Z de longitud al menos 1

`(\d\d\d\d)` encuentra lo mismo que `\d{4}` que son cadenas de cuatro dígitos

`(?P<num>\d\d\d\d)` encuentra  cadenas de cuatro dígitos y etiqueta a ese grupo como "num"


### ¡Muchas gracias por su atención!